In [ ]:
!pip install albumentations

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/Bakis/cnnlstm/split.zip" -d "/content/dataset"

In [ ]:
import argparse
import yaml
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import tensorflow as tf
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, GlobalAveragePooling1D
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from google.colab.patches import cv2_imshow
from tensorflow.keras import backend as K
import random
import albumentations as A

In [ ]:
train_dir = '/content/dataset/training'
test_dir = '/content/dataset/test'
val_dir = '/content/dataset/validation'

In [ ]:
grab_frames = 30
target_shape = 190

In [ ]:
def process_dataset(train_dir, set_size, augment=False):
    X_data = np.zeros((set_size, grab_frames, target_shape, target_shape, 3), dtype=np.uint8)
    y_temp = np.zeros((set_size,), dtype=object)

    index = 0
    random.seed(7)

    transform = A.ReplayCompose([
      A.HorizontalFlip(p=0.5),
      A.MotionBlur(blur_limit=5, p=0.3),
      A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.02, rotate_limit=10, p=0.5),
      A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.4),
      A.CLAHE(clip_limit=2, p=0.3),
      A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.3),
      A.ISONoise(color_shift=(0.01, 0.03), intensity=(0.1, 0.3), p=0.3),
    ])

    data = []

    source_directory = os.path.abspath(train_dir)
    sub_folders = os.listdir(source_directory)

    for folder in sub_folders:
        folder_path = os.path.join(source_directory, folder)
        files = os.listdir(folder_path)

        for file in files:

            video_path = os.path.join(folder_path, file)
            video = cv2.VideoCapture(video_path)
            frames = []
            framesAug = []
            count = 1
            lastFrame = None
            skip = False

            while count <= grab_frames:
                ret, frame = video.read()
                if not ret:
                    break

                if skip == True:
                  skip = not skip
                  continue

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.resize(frame, (target_shape, target_shape))
                lastFrame = frame

                if augment != True:
                  frames.append(frame)

                count += 1

                if augment:
                  if data == []:
                    data = transform(image=frame)
                    framesAug.append(data['image'])
                  else:
                    augImg = A.ReplayCompose.replay(data['replay'], image=frame)
                    framesAug.append(augImg['image'])

                skip = not skip

            video.release()
            cv2.destroyAllWindows()

            if augment != True:
              X_data[index] = np.array(frames)
              y_temp[index] = folder
              index += 1
            if augment:
              X_data[index] = np.array(framesAug)
              y_temp[index] = folder
              index += 1
              data = []
            print(index)


    return X_data, y_temp

In [ ]:
X_train, y_train = process_dataset(train_dir, 2100, True)

In [ ]:
X_val, y_val = process_dataset(val_dir, 200, False)

In [ ]:
lb = LabelBinarizer()

lb.fit(y_train)
y_hot_train = lb.transform(y_train)

lb.fit(y_val)
y_hot_val = lb.transform(y_val)

In [ ]:
lb = LabelBinarizer()
lb.fit(y_test)
y_hot_test = lb.transform(y_test)

In [ ]:
model_path = "/content/drive/MyDrive/Bakis/cnnlstm/model.h5"
rnns = 97
hidden_size = 113
lr = 0.001
classes_num = 10
batch_size = 64
epochs = 100

In [ ]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger("model_history_log.csv", append=True)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Flatten, Dropout, LSTM, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(target_shape, target_shape, 3))
base_model.trainable = False

cnn = Sequential()
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dropout(0.5))

model = Sequential()
model.add(TimeDistributed(cnn, input_shape=(grab_frames, target_shape, target_shape, 3)))
model.add(LSTM(rnns, return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(hidden_size, activation='relu')))
model.add(Dropout(0.5))
model.add(GlobalAveragePooling1D(name="globale"))
model.add(Dense(classes_num, activation="softmax", name="last"))

adam = Adam(learning_rate=lr)

model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)


16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
!pip install keras-tuner --upgrade

In [ ]:
import keras_tuner as kt

In [ ]:
from tensorflow.keras.applications import EfficientNetB0

base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(int(target_shape), int(target_shape), 3))
base_model.trainable = False

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Flatten, Dropout, LSTM, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

In [ ]:
from keras import backend as K

In [ ]:
import keras_tuner as kt

def build_model(hp):
    K.clear_session()

    cnn = Sequential()
    cnn.add(base_model)
    cnn.add(Flatten())
    cnn.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=1, step=0.1)))

    model = Sequential()
    model.add(TimeDistributed(cnn, input_shape=(grab_frames, target_shape, target_shape, 3)))
    model.add(LSTM(hp.Int('lstm_units', min_value=1, max_value=256, step=16), return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=1, step=0.1)))
    model.add(TimeDistributed(Dense(hp.Int('dense_units', min_value=1, max_value=256, step=16), activation='relu')))
    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=1, step=0.1)))
    model.add(GlobalAveragePooling1D(name="globale"))
    model.add(Dense(classes_num, activation="softmax", name="last"))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(hp.Float('lr', 1e-5, 1e-2, sampling='LOG')), metrics=["accuracy"])
    return model

tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=50, factor=3, directory='/content/search', project_name='hyper')
tuner.search(X_train, y_hot_train, validation_data=(X_val, y_hot_val), epochs=50, callbacks=[early_stopping, reduce_lr])

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
history = model.fit(x=X_train,
                    y=y_hot_train,
                    batch_size=int(batch_size),
                    epochs=int(epochs),
                    shuffle=True,
                    validation_data=(X_val, y_hot_val),
                    callbacks=[reduce_lr, csv_logger, early_stopping])

hist = pd.DataFrame(history.history)
plot = hist.plot()
plot.figure.savefig("/content/drive/MyDrive/Bakis/cnnlstm/train.jpg")

model.save(model_path)

print("Learning Curves saved in Report Folder")
print("Model saved to: ", model_path)

In [ ]:
model.summary()

In [ ]:
model.load_model('/content/drive/MyDrive/Bakis/cnnlstm')

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
model.evaluate(x=X_test, y=y_hot_test)

In [ ]:
predictions = model.predict(X_test)